# Datatón Anticurrupción 2021

In [ ]:
###Instalar módulos necesarios
!pip install sklearn
!pip install pandas
!pip install matplotlib
!pip install seaborn

### Carga de archivos  Json

In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
with open('declaraciones.json', encoding="utf8") as file:
    data1 = json.load(file)
declaraciones = pd.json_normalize(data1)
declaraciones.to_csv("declaraciones.csv")

In [9]:
with open('Sistema3Servidores.json', encoding="utf8") as file:
    data2 = json.load(file)
sanciones = pd.json_normalize(data2)
sanciones.to_csv("sanciones.csv")

In [10]:
with open('SistemaS2.json', encoding="utf8") as file:
    data3 = json.load(file)
contratos = pd.json_normalize(data3)
contratos.to_csv("contratos.csv")

### Tratamiento de la base con funcionarios sancionados

In [11]:
sanciones['sancion'] = sanciones['tipoSancion'].apply(lambda x: ','.join(map(str, x)))
sanciones['sancion'] = sanciones['sancion'].map({"{'clave': 'SE', 'valor': 'SANCIÓN ECONÓMICA', 'descripcion': ''}":'SANCIÓN ECONÓMICA',
                                                 "{'clave': 'IRSC', 'valor': 'INDEMNIZACIÓN RESARCITORIA', 'descripcion': ''}":'INDEMNIZACIÓN RESARCITORIA',
                                                 "{'clave': 'I', 'valor': 'INHABILITADO', 'descripcion': ''}":'INHABILITADO',
                                                 "{'clave': 'M', 'valor': 'MULTADO', 'descripcion': ''}":"MULTADO",
                                                 "{'clave': 'O', 'valor': 'OTRO', 'descripcion': 'LEVANTAMIENTO DE ACTA ADMINISTRATIVA'}":"ACTA ADMINISTRATIVA",
                                                 "{'clave': 'S', 'valor': 'SUSPENSION DE EMPLEO, CARGO O COMISION', 'descripcion': ''}":"SUSPENSION",
                                                 "{'clave': 'D', 'valor': 'DESTITUCION DEL EMPLEO CARGO O COMISION', 'descripcion': 'DESPIDO'}":"DESTITUCION",
                                                 "{'clave': 'S', 'valor': 'SUSPENSIÓN DEL EMPLEO, CARGO O COMISIÓN'}":"SUSPENSIÓN"
                                                })

In [12]:
sanciones=sanciones[["expediente", "autoridadSancionadora", "sancion", "causaMotivoHechos", "institucionDependencia.nombre",
                     "servidorPublicoSancionado.curp", "servidorPublicoSancionado.genero.valor", 
                     "servidorPublicoSancionado.puesto", "tipoFalta.valor", "multa.monto", 
                     "inhabilitacion.plazo"
                    ]]

In [13]:
sanciones.rename(columns={'servidorPublicoSancionado.curp': 'curp'}, inplace=True)
sanciones.rename(columns={'causaMotivoHechos': 'causa'}, inplace=True)
sanciones.rename(columns={'institucionDependencia.nombre': 'dependencia'}, inplace=True)
sanciones.rename(columns={'servidorPublicoSancionado.genero.valor': 'genero'}, inplace=True)
sanciones.rename(columns={'servidorPublicoSancionado.puesto': 'puesto'}, inplace=True)
sanciones.rename(columns={'tipoFalta.valor': 'falta'}, inplace=True)
sanciones.rename(columns={'multa.monto': 'multa'}, inplace=True)
sanciones.rename(columns={'inhabilitacion.plazo': 'inhabilitacion'}, inplace=True)

In [14]:
sanciones.to_csv("sanciones_clear.csv")

### Guardamos CSV especiales para multas e inahilitaciones

In [15]:
sanciones_multas = sanciones.dropna(subset=["multa"])
sanciones_multas.to_csv("sanciones_multas.csv")

In [16]:
sanciones_inhabilitacion = sanciones.dropna(subset=["inhabilitacion"])
sanciones_inhabilitacion.to_csv("sanciones_inhabilitacion.csv")

### Tratamiento de la base con declaraciones

In [17]:
declaraciones2 = declaraciones[["metadata.institucion", "metadata.declaracionCompleta", 
                                "declaracion.situacionPatrimonial.datosGenerales.curp", 
                                "declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad",
                               "declaracion.situacionPatrimonial.experienciaLaboral.experiencia",
                               "declaracion.situacionPatrimonial.ingresos.ingresoAnualNetoDeclarante.valor",
                               "declaracion.situacionPatrimonial.ingresos.ingresoAnualNetoDeclarante.moneda"]]

In [18]:
declaraciones2.rename(columns={'metadata.institucion': 'institucion'}, inplace=True)
declaraciones2.rename(columns={'metadata.declaracionCompleta': 'completa'}, inplace=True)
declaraciones2.rename(columns={'declaracion.situacionPatrimonial.datosGenerales.curp': 'curp'}, inplace=True)
declaraciones2.rename(columns={'declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad': 'escolaridad'}, inplace=True)
declaraciones2.rename(columns={'declaracion.situacionPatrimonial.experienciaLaboral.experiencia': 'experiencia'}, inplace=True)
declaraciones2.rename(columns={'declaracion.situacionPatrimonial.ingresos.ingresoAnualNetoDeclarante.valor': 'ingreso'}, inplace=True)
declaraciones2.rename(columns={'declaracion.situacionPatrimonial.ingresos.ingresoAnualNetoDeclarante.moneda': 'moneda'}, inplace=True)

C:\Users\syste\anaconda3\envs\Dataton\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [19]:
df_merged = pd.merge(sanciones, 
                     declaraciones2,
                     on=['curp'])

In [20]:
df_merged.to_csv("clusters.csv")

![cat](https://lh3.googleusercontent.com/proxy/r0NaCbk-E3YHSbtdEKDy9wUM_PL5Gwoou3kk2Ym0ciY6MWiZNwbZnV1DR-5MLHprhQOp-2y5fAd20VDTKXO9zhzFYLknMjF12Ugz92TB7ECbURs5O4tokJvDNZPt_7oHcaVpGwALyg)